In [ ]:
!pip install streamlit google-generativeai pyngrok


In [ ]:
%%writefile app.py
import streamlit as st
import google.generativeai as genai

GOOGLE_API_KEY = "your gemini api key"
genai.configure(api_key=GOOGLE_API_KEY)

def call_gemini(system_prompt, user_prompt, model="gemini-2.5-flash"):
    combined_prompt = f"{system_prompt}\n\nUser Query: {user_prompt}"
    model_instance = genai.GenerativeModel(model_name=model)
    response = model_instance.generate_content(combined_prompt)
    return response.text

def symptom_checker_agent(query):
    system_prompt = """You are a symptom checking assistant. Given patient symptoms, you provide possible causes and suggest next steps."""
    user_prompt = f"Analyze these symptoms: {query}"
    return call_gemini(system_prompt, user_prompt)

def medical_research_agent(query):
    system_prompt = """You are a medical research assistant. Provide detailed and up-to-date medical information about diseases, treatments, or medications."""
    user_prompt = f"Research medical information about: {query}"
    return call_gemini(system_prompt, user_prompt)

def treatment_pricing_agent(query):
    system_prompt = """You are a healthcare pricing advisor. Provide information about treatment costs, insurance options, and price comparisons."""
    user_prompt = f"Provide pricing information for: {query}"
    return call_gemini(system_prompt, user_prompt)

def routing_agent(query):
    routing_system_prompt = """You are a healthcare routing agent. Classify queries into exactly one of the following:
    - SYMPTOM_CHECKER: for symptom analysis and possible diagnoses
    - MEDICAL_RESEARCH: for detailed medical information or research about diseases and treatments
    - TREATMENT_PRICING: for pricing information about treatments or healthcare services

    Respond ONLY with one of these labels."""

    routing_user_prompt = f"Classify this healthcare query: {query}"
    task_type = call_gemini(routing_system_prompt, routing_user_prompt).strip()

    if "SYMPTOM_CHECKER" in task_type:
        return "[Symptom Checker Agent]\n" + symptom_checker_agent(query)
    elif "MEDICAL_RESEARCH" in task_type:
        return "[Medical Research Agent]\n" + medical_research_agent(query)
    elif "TREATMENT_PRICING" in task_type:
        return "[Treatment Pricing Agent]\n" + treatment_pricing_agent(query)
    else:
        return "Sorry, I could not route your query. Please try rephrasing."

st.title("Healthcare Support System (Powered by Google Gemini)")
st.write("Enter your healthcare query and get answers from our smart multi-agent system.")

user_input = st.text_area("Enter your query here:")

if st.button("Get Response"):
    if user_input.strip():
        with st.spinner("Processing..."):
            response = routing_agent(user_input)
        st.markdown("### Response:")
        st.write(response)
    else:
        st.error("Please enter a query to continue.")


In [ ]:
!ngrok config add-authtoken your ngrok auth token


In [ ]:
from pyngrok import ngrok, conf

# Set your ngrok auth token here
conf.get_default().auth_token = "your auth token"

# Kill any existing tunnels
ngrok.kill()

# Open a tunnel on port 8501 (Streamlit default)
public_url = ngrok.connect(8501)
print(f"Ngrok tunnel opened at: {public_url}")


In [ ]:
!streamlit run app.py &